In [7]:
import pandas as pd

#bee_data_path = '/content/0458DF23-A6F2-3B7F-975B-698396FB1BD6.csv'
bee_data_path= 'https://raw.githubusercontent.com/ModelEarth/bee-data/refs/heads/main/honeybee%20population%20data.csv'

bee_data = pd.read_csv(bee_data_path)

# Step 1: Remove entries with Value as '(D)'
bee_data = bee_data[bee_data['Value'] != '(D)']
bee_data['County']=bee_data['County'].str.lower()

# Convert the Value column to numeric
bee_data['Value'] = pd.to_numeric(bee_data['Value'], errors='coerce')

bee_data.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2022,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,autauga,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,258.0,34.6
1,CENSUS,2022,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,bullock,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,48.0,34.6
2,CENSUS,2022,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,dallas,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,629.0,34.6
3,CENSUS,2022,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,elmore,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,305.0,34.6
4,CENSUS,2022,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,greene,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,40.0,34.6


In [9]:
import requests
import pandas as pd

# API URL
url = "https://api.census.gov/data/2023/geoinfo?get=GEO_ID,NAME,AREALAND,AREAWATR&for=county:*"
# params = {
#     "get": "GEO_ID,NAME,AREALAND,AREAWATR",
#     "for": "county:*"
# }

# Make the API request
response = requests.get(url)

In [10]:
# Check the response status
if response.status_code == 200:
    # Load the response data
    data = response.json()

    # Create a DataFrame from the data
    df = pd.DataFrame(data[1:], columns=data[0])

    # Convert AREALAND and AREAWATR to numeric and calculate total area (in square kilometers)
    df["AREALAND"] = pd.to_numeric(df["AREALAND"])
    df["AREAWATR"] = pd.to_numeric(df["AREAWATR"])
    df["Total_Area_km2"] = (df["AREALAND"] + df["AREAWATR"]) / 1e6

    # Split NAME into 'county' and 'state' (convert to lowercase)
    df[["county", "state"]] = df["NAME"].str.split(", ", expand=True)
    df["county"] = df["county"].str.replace(' County', '', regex=False).str.lower()
    df["state"] = df["state"].str.lower()

    # Keep only the last 5 digits of GEO_ID as FIPS
    df["FIPS"] = df["GEO_ID"].str[-5:]

    # Keep only the desired columns
    df = df[["FIPS", "county", "state", "Total_Area_km2"]]

    # Display the resulting DataFrame
    print(df)
else:
    print(f"Failed to fetch data. Status Code: {response.status_code}, Response: {response.text}")


       FIPS               county        state  Total_Area_km2
0     01001              autauga      alabama     1565.308997
1     01003              baldwin      alabama     5250.612251
2     01005              barbour      alabama     2342.683364
3     01007                 bibb      alabama     1621.761015
4     01009               blount      alabama     1685.119381
...     ...                  ...          ...             ...
3217  72145  vega baja municipio  puerto rico      176.572672
3218  72147    vieques municipio  puerto rico      683.734210
3219  72149   villalba municipio  puerto rico       95.921207
3220  72151    yabucoa municipio  puerto rico      215.597702
3221  72153      yauco municipio  puerto rico      176.997181

[3222 rows x 4 columns]


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:

# # Step 2: Filter for the year 2017
# bee_data_filtered = bee_data[(bee_data['Year'] == 2017)]


# # Select only relevant columns for the join
# us_me_training_filtered = df[['county', 'FIPS', 'Total_Area_km2']]

# # Step 4: Perform an inner join on County name

# merged_data = pd.merge(
#     bee_data_filtered,
#     df,
#     left_on=['County', 'State']
#     right_on=['county', 'state']
#     how='inner'
# )

# # Step 5: Create the bee population density column
# merged_data['bee_population_density'] = merged_data['Value'] / merged_data['Total_Area_km2']

# # Step 6: Create the target column based on the top 20% bee density
# density_threshold = merged_data['bee_population_density'].quantile(0.8)
# merged_data['target'] = (merged_data['bee_population_density'] > density_threshold).astype(int)
# merged_data[['FIPS', 'target']]

,FIPS,target
0,01001,0
1,01011,0
2,01047,0
3,05039,0
4,19049,0
...,...,...
13753,56035,0
13754,16081,0
13755,30099,0
13756,56039,0


In [ ]:
# # Step 2: Filter for the year 2017
# bee_data_filtered = bee_data[(bee_data['Year'] == 2017)]
# bee_data_filtered['State']=bee_data['State'].str.lower()


# # Select only relevant columns for the join
# us_me_training_filtered = df[['county', 'FIPS', 'Total_Area_km2']]

# # Step 4: Perform an inner join on County name and state

# merged_data = pd.merge(
#     bee_data_filtered,
#     df,
#     left_on= ['County', 'State'],
#     right_on=['county', 'state'],
#     how='inner'
# )

# # Step 5: Create the bee population density column
# merged_data['bee_population_density'] = merged_data['Value'] / merged_data['Total_Area_km2']

# # Step 6: Create the target column based on the top 20% bee density
# density_threshold = merged_data['bee_population_density'].quantile(0.8)
# merged_data['target'] = (merged_data['bee_population_density'] > density_threshold).astype(int)
# merged_data[['FIPS', 'target']]

<ipython-input-7-8a3d35cbf541>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bee_data_filtered['State']=bee_data['State'].str.lower()


,FIPS,target
0,01001,0
1,01011,0
2,01047,0
3,01051,0
4,01063,0
...,...,...
2742,56031,0
2743,56023,0
2744,56035,0
2745,56039,0


In [11]:
# Step 2: Filter for the year 2017
bee_data_filtered = bee_data[(bee_data['Year'] == 2017)]
bee_data_filtered['State']=bee_data['State'].str.lower()


# Select only relevant columns for the join
us_me_training_filtered = df[['county', 'FIPS', 'Total_Area_km2']]

# Step 4: Perform an inner join on County name and state

merged_data = pd.merge(
    bee_data_filtered,
    df,
    left_on= ['County', 'State'],
    right_on=['county', 'state'],
    how='inner'
)

# Step 5: Create the bee population density column
merged_data['bee_population_density'] = merged_data['Value'] / merged_data['Total_Area_km2']

# Step 6: Create the target column based on the top 20% bee density
density_threshold = merged_data['bee_population_density'].quantile(0.8)
merged_data['target'] = (merged_data['bee_population_density'] > density_threshold).astype(int)
merged_data[['FIPS', 'target']]

<ipython-input-11-af50d433395c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bee_data_filtered['State']=bee_data['State'].str.lower()


,FIPS,target
0,01001,0
1,01011,0
2,01047,0
3,01051,0
4,01063,0
...,...,...
2742,56031,0
2743,56023,0
2744,56035,0
2745,56039,0


In [12]:
merged_data[['FIPS', 'target']].to_csv('Target.csv')

In [ ]:
import base64
import requests

# === CONFIGURATION ===
GITHUB_TOKEN = ""

REPO_OWNER = "ModelEarth"
REPO_NAME = "bee-data"
BRANCH = "main"
FILE_PATH = "Target.csv"
REPO_FILE_PATH = "targets/bees-targets-top-20-percent.csv"
# === HEADERS ===
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json" 
}

with open(FILE_PATH, "rb") as f:
    content = f.read()
    # content_bytes = content.encode('utf-8')
    base64_content = base64.b64encode(content).decode("utf-8")


In [14]:

url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{REPO_FILE_PATH}?ref={BRANCH}"
response = requests.get(url, headers=headers)
print(response.json())
file_exists = response.status_code == 200


{'name': 'bees-targets-top-20-percent.csv', 'path': 'targets/bees-targets-top-20-percent.csv', 'sha': '0cb809e710711bdd4c2542b91034cd411ccf7b88', 'size': 24478, 'url': 'https://api.github.com/repos/ModelEarth/bee-data/contents/targets/bees-targets-top-20-percent.csv?ref=main', 'html_url': 'https://github.com/ModelEarth/bee-data/blob/main/targets/bees-targets-top-20-percent.csv', 'git_url': 'https://api.github.com/repos/ModelEarth/bee-data/git/blobs/0cb809e710711bdd4c2542b91034cd411ccf7b88', 'download_url': 'https://raw.githubusercontent.com/ModelEarth/bee-data/main/targets/bees-targets-top-20-percent.csv', 'type': 'file', 'content': '...\n', 'encoding': 'base64', '_links': {'self': 'https://api.github.com/repos/ModelEarth/bee-data/contents/targets/bees-targets-top-20-percent.csv?ref=main', 'git': 'https://api.github.com/repos/ModelEarth/bee-data/git/blobs/0cb809e710711bdd4c2542b91034cd411ccf7b88', 'html': 'https://github.com/ModelEarth/bee-data/blob/main/targets/bees-targets-top-20-per

In [15]:

payload = {
    "message": "Upload or update CSV file via API",
    "content": base64_content,
    "branch": BRANCH
}

if file_exists:
    # Extract SHA if file exists
    payload["sha"] = response.json()["sha"]

In [16]:
import json

put_response = requests.put(url, headers=headers, json=payload)

if put_response.status_code in [200, 201]:
    print("✅ File uploaded successfully!")
    print("📄 File URL:", put_response.json()["content"]["html_url"])
else:
    print("❌ Failed to upload file")
    print("Status Code:", put_response.status_code)
    print("Response:", put_response.json())

✅ File uploaded successfully!
📄 File URL: https://github.com/ModelEarth/bee-data/blob/main/targets/bees-targets-top-20-percent.csv
